##  迁移学习 (keras)

- 从哪里迁移？
    - 源网络：keras内置的基于imageNet数据集训练好的[网络](https://keras.io/applications/) (包括Xception,VGG16,VGG19,ResNet50,InceptionV3等等)
- 迁移到哪里？
    - 目标数据集：cifar10
- 怎么迁移？
    - 去除源网络最后一层（原来的1000个类别的输出层）
    - 加上神经元个数为10的输出层（cifar10有10个类别）
    - 加载前面所有层的源网络上训练好的权重
    - 随机初始化最后一层权重
    - 训练最后一层
 

In [11]:
import keras
from keras.applications.resnet50 import ResNet50
import keras.datasets.cifar10 as cifar10
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential,Model
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"


### 加载 cifar10 数据集
首次加载时会自动从网络自动下载该数据集，非首次加载时会自动从本地读取

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


- 训练数据集一共有50000张图片
- 测试数据集一共有10000张图片
- 图片大小 32 * 32 * 3 

In [3]:
print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


### 设置模型相关参数

In [ ]:
category_num = 10
epochs = 10
batch_size = 256

### one-hot 编码

In [4]:
y_train = np_utils.to_categorical(y_train, category_num)
y_test = np_utils.to_categorical(y_test, category_num)

### 定义图片进行resize预处理方法和产生一个batch数据的方法

In [ ]:
def preprocess(img, target_size):
    img = image.array_to_img(img, scale=False)
    img = img.resize(target_size)
    img = image.img_to_array(img)
    return img

def my_generator(X, Y, batch_size=32, target_size=(224,224)):
    indexs = list(range(X.shape[0]))
    while True:
        np.random.shuffle(indexs)
        for i in range(0, len(indexs), batch_size):
            ge_batch_x = [preprocess(img,target_size) for img in X[i:i+batch_size]]
            ge_batch_y = Y[i:i+batch_size]
            ge_batch_x = np.array(ge_batch_x)
            ge_batch_y = np.array(ge_batch_y)
            yield ge_batch_x, ge_batch_y

### 创建网络

In [5]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(category_num, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

### 冻结原始网络层的全部权重不做训练

In [6]:
for layer in base_model.layers:
    layer.trainable = False

### 训练网络

In [18]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.0001), metrics=['accuracy'])
model.fit_generator(
        my_generator(x_train, y_train, batch_size=batch_size, target_size=(224,224)),
        steps_per_epoch=len(x_train)/batch_size,
        epochs=epochs,
        validation_data=my_generator(x_test, y_test, batch_size=batch_size, target_size=(224,224)),
        validation_steps=len(x_test)/batch_size,
        verbose=1)

Epoch 1/10
196/195 [==============================] - 234s 1s/step - loss: 1.2684 - acc: 0.5968 - val_loss: 1.0591 - val_acc: 0.6608
Epoch 2/10
196/195 [==============================] - 234s 1s/step - loss: 0.9555 - acc: 0.6905 - val_loss: 0.8857 - val_acc: 0.7049
Epoch 3/10
196/195 [==============================] - 234s 1s/step - loss: 0.8378 - acc: 0.7231 - val_loss: 0.8100 - val_acc: 0.7282
Epoch 4/10
196/195 [==============================] - 234s 1s/step - loss: 0.7730 - acc: 0.7434 - val_loss: 0.7638 - val_acc: 0.7412
Epoch 5/10
196/195 [==============================] - 235s 1s/step - loss: 0.7302 - acc: 0.7573 - val_loss: 0.7320 - val_acc: 0.7522
Epoch 6/10
196/195 [==============================] - 235s 1s/step - loss: 0.6988 - acc: 0.7665 - val_loss: 0.7085 - val_acc: 0.7592
Epoch 7/10
196/195 [==============================] - 235s 1s/step - loss: 0.6744 - acc: 0.7745 - val_loss: 0.6901 - val_acc: 0.7651
Epoch 8/10
196/195 [==============================] - 235s 1s/step - 